In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=3)
    KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 3)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(4)
TTC.set_alice_transmission_channel(3)
# set bob channels
TTC.set_bob_reflection_channel(2)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()


In [5]:
#TTC.displayCountTraces()
#TTC.displayCoincidenceTraces(coincidence_window_SI=0.4e-9)

In [6]:
# TODO
# coincidence channel handling and destruction
# 
#TTC.coincidences_vchannels.getChannels()

In [7]:
#KMC.rotate_simulataneously(0, 45)

In [8]:
#TTC.performDelayAdjustment(True)

In [9]:
idealAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> + |VV> initial state
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9)



corr[a,b] = 0.9159663865546218
	N[|T,T>]=585
	N[|T,R>]=5
	N[|R,T>]=25
	N[|R,R>]=99

corr[a,B] = 0.7595129375951294
	N[|T,T>]=491
	N[|T,R>]=13
	N[|R,T>]=66
	N[|R,R>]=87

corr[A,b] = 0.6448598130841121
	N[|T,T>]=395
	N[|T,R>]=62
	N[|R,T>]=33
	N[|R,R>]=45

corr[A,B] = 0.06951871657754011
	N[|T,T>]=170
	N[|T,R>]=61
	N[|R,T>]=113
	N[|R,R>]=30

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.250820420656323


# OSC Server Setup


In [10]:
my_ip = '192.168.0.2'
my_port = 7401 

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=7400, name='Clemens')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [11]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Myself, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Myself


Received measurement: (1, 2, 3, 4)


In [12]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
CHSH_angles = [0, -22.5, -11.5, 11.5]

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    while performance_running:
        theta_a = np.random.choice(CHSH_angles[0:2])
        theta_b = np.random.choice(CHSH_angles[2:4])
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        resultClemensFormat = [1 if theta_a == CHSH_angles[0] else 2, 1 if theta_b == CHSH_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [24]:
stop_performance()

In [23]:
start_performance()

Measurement [2, 2, 1, 1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
